In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import linear_model
    
pd.set_option('display.max_columns', 500)

In [137]:
## Lecture des données

data = pd.read_csv('AmesHousing.tsv', sep='\t')

In [187]:
print(data.shape)

print(len(str(data.shape))*'--')

print(data.dtypes.value_counts())

data.head() 



(2930, 82)
--------------------
object     43
int64      28
float64    11
dtype: int64


,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [139]:
def transform_features(df):
    return df

In [140]:
def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]

In [141]:
def train_and_test(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop("SalePrice")
    
    ## Entraînement du modèle de regression
    
    train_features = train[features]
    train_target = train["SalePrice"]
    
    test_features = test[features]
    test_target = test['SalePrice']
    
    lr = linear_model.LinearRegression()
    lr.fit(train_features, train_target)
    
    ## prédiction:
    
    predictions = lr.predict(test_features)
    mse = mean_squared_error(test_target, predictions)
    rmse = np.sqrt(mse)
    
    
    return rmse
    

In [142]:
transform_df = transform_features(data)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)
rmse

57088.25161263909

## RMSE_1 = 57088.25161263909

## Amélioration du modèle
### . Traitement des caractéristiques:

#### . Suppression de toutes les colonnes avec >= 5% des valeurs manquantes pour le moment

#### . Suppression de toutes les colonnes textes contenant >= 1 valeur manquante pour le moment

#### . Remplacement des valeurs manquantes dans des colonnes numériques par les valeurs les plus fréquentes de la colonne

In [143]:
# calcul du nombre de valeur manquante
num_missing = data.isnull().sum()

## filtrage de sur les colonnes contenant >= 5% des NA

missing_cols = num_missing[(num_missing > (len(data)/20))].sort_values()

## Maintenant suppression de ces colonnes 
data = data.drop(missing_cols.index, axis = 1)

## vérification: 
data.shape

(2930, 71)

In [144]:
## On va supprimer toutes les colonnes texte contenant au moins une valeur manquante:

# On sélectionne les colonnes contenant les objets(texte)
text_missing = data.select_dtypes(include=['object']).isnull().sum().sort_values(
ascending = False)

## On filtre data sur les colonnes contenant au moins une NA

missing_cols_2 = text_missing[text_missing >= 1]

# puis on supprime:
data = data.drop(missing_cols_2.index, axis = 1)

## On peut vérifier à nouveau le nonbre de colonnee restantes
data.shape

(2930, 64)

In [145]:
### Remplecement des valeurs numérique manquantes par la val la plus fréquente

## calcul de val manquantes colonne/colonne

num_missing = data.select_dtypes(include=['int', 'float']).isnull().sum()

f = num_missing[(num_missing < len(data)/20) &
               (num_missing >= 1)].sort_values()
f

BsmtFin SF 1       1
BsmtFin SF 2       1
Bsmt Unf SF        1
Total Bsmt SF      1
Garage Cars        1
Garage Area        1
Bsmt Full Bath     2
Bsmt Half Bath     2
Mas Vnr Area      23
dtype: int64

In [146]:
### Maintenant il faut trouver le mode(valeur la plus fréquente ou commune de chaque colonne)

cols_mode = data[f.index].mode()
cols_mode_to_dict = cols_mode.to_dict(orient='records')
remplacement_values = cols_mode_to_dict[0] ## on a les modes de chaque colonne
remplacement_values

{'BsmtFin SF 1': 0.0,
 'BsmtFin SF 2': 0.0,
 'Bsmt Unf SF': 0.0,
 'Total Bsmt SF': 0.0,
 'Garage Cars': 2.0,
 'Garage Area': 0.0,
 'Bsmt Full Bath': 0.0,
 'Bsmt Half Bath': 0.0,
 'Mas Vnr Area': 0.0}

In [147]:
## remplacement des valeurs manquantes par les modes...

data = data.fillna(remplacement_values)

## vérification
data.isnull().sum().value_counts() ## plus aucune valeur manquante

0    64
dtype: int64

### Quelles nouvelles caractéristiques pouvons-nous créer pour mieux capturer les informations contenues dans certaines caractéristiques?? 

In [148]:
## Différence entre année de vente et de construction:
years_sold = data['Yr Sold'] - data['Year Built']

years_sold[years_sold < 0] ## est-ce que le bien a été vendu avant sa construction? 


2180   -1
dtype: int64

In [149]:
### compraison de la date de vente et de rénovation

years_since_remod = data['Yr Sold'] - data['Year Remod/Add']
years_since_remod[years_since_remod < 0]

1702   -1
2180   -2
2181   -1
dtype: int64

In [150]:
### Ces caractéristiques sont intéréssants, on va créer deux
## nouvelles colonnes avec ces deux derniers carart 

data['Years Before Sale'] = years_sold
data['Years Since Remod'] = years_since_remod

In [151]:
## On va supprimer les valeurs négatives de ces nouvelles colonnes
data = data.drop([1702, 2180, 2181], axis = 0)


In [152]:
## supression de l'année d'origine
data = data.drop(['Year Built', 'Year Remod/Add'], axis = 1)

#### Suppression des colonnes non importantes au ML et celles qui génèrent une fuite des données au sujet de la vente finale(Cf documentation dataset) 

In [153]:
## Cols non utiles
data = data.drop(['PID', 'Order'], axis = 1)

## cols qui font fuiter...
data = data.drop(['Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis = 1)

In [154]:
data.head()

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Garage Cars,Garage Area,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remod
0,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,Hip,CompShg,BrkFace,Plywood,112.0,TA,TA,CBlock,639.0,0.0,441.0,1080.0,GasA,Fa,Y,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,2.0,528.0,P,210,62,0,0,0,0,0,215000,50,50
1,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,Gable,CompShg,VinylSd,VinylSd,0.0,TA,TA,CBlock,468.0,144.0,270.0,882.0,GasA,TA,Y,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,1.0,730.0,Y,140,0,0,0,120,0,0,105000,49,49
2,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,Hip,CompShg,Wd Sdng,Wd Sdng,108.0,TA,TA,CBlock,923.0,0.0,406.0,1329.0,GasA,TA,Y,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,1.0,312.0,Y,393,36,0,0,0,0,12500,172000,52,52
3,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,Hip,CompShg,BrkFace,BrkFace,0.0,Gd,TA,CBlock,1065.0,0.0,1045.0,2110.0,GasA,Ex,Y,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,2.0,522.0,Y,0,0,0,0,0,0,0,244000,42,42
4,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,Gable,CompShg,VinylSd,VinylSd,0.0,TA,TA,PConc,791.0,0.0,137.0,928.0,GasA,Gd,Y,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,2.0,482.0,Y,212,34,0,0,0,0,0,189900,13,12


### MIS A JOUR DES FONCTIONS DE PEDICTION EN PRENANT EN COMPTE LE TRAITEMENT DES CARACTERISTIQUES

In [155]:
def transform_features_2(data, percent_missing=0.05):
    
    # calcul du nombre de valeur manquante
    num_missing = data.isnull().sum()

## filtrage de sur les colonnes contenant >= 5% des NA

    missing_cols = num_missing[(num_missing > (len(data)*percent_missing))].sort_values()

## Maintenant suppression de ces colonnes 
    data = data.drop(missing_cols.index, axis = 1)
    
    #------------------------
    
    # On sélectionne les colonnes contenant les objets(texte)
    text_missing = data.select_dtypes(include=['object']).isnull().sum().sort_values(
    ascending = False)

## On filtre data sur les colonnes contenant au moins une NA

    missing_cols_2 = text_missing[text_missing >= 1]

# puis on supprime:
    data = data.drop(missing_cols_2.index, axis = 1)
    
    #---------------------------
    ### Remplecement des valeurs numérique manquantes par la val la plus fréquente

## calcul de val manquantes colonne/colonne

    num_missing = data.select_dtypes(include=['int', 'float']).isnull().sum()

    fixable_numeric_cols = num_missing[(num_missing < len(data)*percent_missing) &
               (num_missing >= 1)].sort_values()
    
    #-----------------
    ### Maintenant il faut trouver le mode(valeur la plus fréquente ou commune de chaque colonne)

    cols_mode = data[f.index].mode()
    cols_mode_to_dict = cols_mode.to_dict(orient='records')
    remplacement_values = cols_mode_to_dict[0] ## on les modes de chaque colonne

    ## remplacement des valeurs manquantes par les modes...

    data = data.fillna(remplacement_values)
    
    #-------------------
    years_sold = data['Yr Sold'] - data['Year Built']
    years_since_remod = data['Yr Sold'] - data['Year Remod/Add']
    
    data['Years Before Sale'] = years_sold
    data['Years Since Remod'] = years_since_remod
    
    data = data.drop([1702, 2180, 2181], axis = 0)
    data = data.drop(['Year Built', 'Year Remod/Add', 
                     'Mo Sold', 'Sale Condition', 'Sale Type', 'Yr Sold'], axis = 1)
    
    return data

In [156]:

def select_features_2(df):
    return df[["Gr Liv Area", "SalePrice"]]

In [183]:
def train_and_test_2(df):
    train = df[:1460]
    test = df[1460:]
    
    numeric_train = train.select_dtypes(include=['integer', 'float'])
    numeric_test = test.select_dtypes(include=['integer', 'float'])
    
    features = numeric_train.columns.drop("SalePrice")
    
    ## Entraînement du modèle de regression
    
    train_features = train[features]
    train_target = train["SalePrice"]
    
    test_features = test[features]
    test_target = test['SalePrice']
    
    lr = linear_model.LinearRegression()
    lr.fit(train_features, train_target)
    
    ## prédiction:
    
    predictions = lr.predict(test_features)
    mse = mean_squared_error(test_target, predictions)
    rmse = np.sqrt(mse)
    
    
    return rmse
    

In [184]:
data = pd.read_csv('AmesHousing.tsv', sep='\t')
transform_data2 = transform_features_2(data)
filtered_data2 = select_features_2(transform_data2)
rmse = train_and_test_2(filtered_data2)
rmse  ### Modèle améliorée 

55275.367312413066

## RMSE_2 = 55275.367312413066 --> Amélioré

### SELECTION DES MEILLEURS CARACTERISTIQUES

In [159]:
## 
numerical_df = transform_data2.select_dtypes(include=['int', 'float'])
numerical_df.head()

,Order,PID,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,TotRms AbvGrd,Fireplaces,Garage Cars,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,Years Before Sale,Years Since Remod
0,1,526301100,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,0,0,1656,1.0,0.0,1,0,3,1,7,2,2.0,528.0,210,62,0,0,0,0,0,215000,50,50
1,2,526350040,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,0,0,896,0.0,0.0,1,0,2,1,5,0,1.0,730.0,140,0,0,0,120,0,0,105000,49,49
2,3,526351010,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,0,0,1329,0.0,0.0,1,1,3,1,6,0,1.0,312.0,393,36,0,0,0,0,12500,172000,52,52
3,4,526353030,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,0,0,2110,1.0,0.0,2,1,3,1,8,2,2.0,522.0,0,0,0,0,0,0,0,244000,42,42
4,5,527105010,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,701,0,1629,0.0,0.0,2,1,3,1,6,1,2.0,482.0,212,34,0,0,0,0,0,189900,13,12


In [160]:
##
abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
abs_corr_coeffs

BsmtFin SF 2         0.006127
Misc Val             0.019273
Order                0.031542
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
MS SubClass          0.085128
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
PID                  0.246389
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qu

In [161]:
## Gardons uniquement les colonnes ayant > 0.4 de coefs de corr

abs_corr_coeffs[abs_corr_coeffs > 0.4]

BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
Years Since Remod    0.534985
Full Bath            0.546118
Years Before Sale    0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64

In [162]:
### supprimons donc le reste, avec des coeff de corr <= 0.4

transform_data2 = transform_data2.drop(abs_corr_coeffs[abs_corr_coeffs <= 0.4].index, axis=1)
transform_data2.shape[1]

39

In [163]:
## colonnes de types dummies à garder 

In [165]:
### liste de colonnes catégoriques à partir de la documentation 

nominal_features = ['PID', 'MS SubClass', 'MS Zoning', 'Street', 'Alley', 'Land Contour', 
                     'Lot Config', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type',
                     'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd',
                     'Mas Vnr Type', 'Foundation', 'Heating', 'Central Air', 'Garage Type', 'Misc Feature',
                     'Sale Type', 'Sale Condition'
    
]

In [166]:
##On vérifie si ces colonnes sont toujours présentes dans notre dataTransform2
transform_cat_cols = []
for col in nominal_features:
    if col in transform_data2.columns:
        transform_cat_cols.append(col)
transform_cat_cols

['MS Zoning',
 'Street',
 'Land Contour',
 'Lot Config',
 'Neighborhood',
 'Condition 1',
 'Condition 2',
 'Bldg Type',
 'House Style',
 'Roof Style',
 'Roof Matl',
 'Exterior 1st',
 'Exterior 2nd',
 'Foundation',
 'Heating',
 'Central Air']

In [167]:
##Comptons les valeurs uniques.

unique_counts = transform_data2[
    transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()

## On fixe une limite (arbritrairement) à 10 valeurs uniques

drop_nouning_cols = unique_counts[unique_counts > 10].index
transform_data2 = transform_data2.drop(drop_nouning_cols, axis=1)


In [169]:
### Sélection des colonnes de textes restantes en les convertissant en dummies

text_cols = transform_data2.select_dtypes(include=['object'])

for col in text_cols:
    transform_data2[col] = transform_data2[col].astype('category')

In [170]:
### création des colonnes factices

transfrom_data2 = pd.concat([
    transform_data2,
    pd.get_dummies(transform_data2.select_dtypes(include=['category']))
], axis = 1)